In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.layers import RNN, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import *

df_2 = pd.read_csv('/content/Patient_result_RNN_10_part2.csv')

vertical_MAE = df_2['MAE'].values
vertical_MSE= np.power(df_2["RMSE"].values,2)
vertical_RMSE= df_2["RMSE"].values
vertical_MRE= df_2["MMRE"].values
vertical_MMRE= df_2["MMRE"].values
vertical_PRED= df_2["PRED"].values

for j in range(60,90):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
      X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
      X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1],1))
      # Define the RNN model
      model = Sequential()
      model.add(SimpleRNN(64))
      model.add(Dense(1))
      # Compile the model
      model.compile(loss='mean_squared_error', optimizer='adam')
      # Train the model
      model.fit(X_train, y_train, epochs=50,batch_size = 32, verbose=0)
      # Make Predictions
      pred = model.predict(X_test)
      df.at[i,'SensorGLU']=pred


  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)
  tf.keras.backend.clear_session()  # Clear TensorFlow session to free up memory
  print("Tensorflow/keras session cleared")


60
1/1 [==============================] - 1s 605ms/step


1/1 [==============================] - 0s 230ms/step


1/1 [==============================] - 0s 242ms/step
Tensorflow/keras session cleared
61
1/1 [==============================] - 0s 185ms/step
Tensorflow/keras session cleared
62
1/1 [==============================] - 0s 138ms/step
Tensorflow/keras session cleared
63
1/1 [==============================] - 0s 133ms/step
Tensorflow/keras session cleared
64
1/1 [==============================] - 0s 135ms/step
Tensorflow/keras session cleared
65
1/1 [==============================] - 0s 134ms/step
Tensorflow/keras session cleared
66
1/1 [==============================] - 0s 153ms/step
Tensorflow/keras session cleared
67
1/1 [==============================] - 0s 142ms/step
Tensorflow/keras session cleared
68
1/1 [==============================] - 0s 144ms/step
Tensorflow/keras session cleared
69
1/1 [==============================] - 0s 145ms/step
Tensorflow/keras session cleared
70
1/1 [==============================] - 0s 191ms/step
Tensorflow/keras session cleared
71
1/1 [================

In [4]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_RNN_10_part3.csv', index=False)

In [5]:
test.shape

(89, 4)

In [6]:
test

,MAE,RMSE,MMRE,PRED
0,238.223854,238.223854,82.716616,0.000000
1,1.703142,1.703142,5.622256,50.000000
2,98.828085,98.828085,76.021604,33.333333
3,37.965950,37.965950,31.638292,25.000000
4,112.848640,112.848640,65.609675,20.000000
...,...,...,...,...
84,172.049603,172.049603,86.024801,9.411765
85,21.874001,21.874001,32.167648,9.302326
86,72.778334,72.778334,69.979167,9.195402
87,206.136173,206.136173,79.897742,9.090909
